In [ ]:
from preprocessing.data_cleaning import get_cleaned_data, split_csv
from preprocessing.encoding import encode_data
from exploration.data_correlations import plot_correlations
from exploration.data_statistics import plot_patientwise_errors, plot_labels_pie_chart, plot_blood_draw_statistics
from utils.utils import log_features

In [ ]:
df = get_cleaned_data()
df = encode_data(df)
log_features(df)

In [ ]:
plot_patientwise_errors(
    df,
    ["A1_RACE", "C1_WAIS3_DIGTOT", "B1_BMI", "RBM_Insulin"],
    ["Race", "WAIS3 Digits Score", "BMI", "Insulin"],
    png="Patientwise Errors"
)

In [ ]:
plot_labels_pie_chart(df, png="Pie Chart of Diagnoses")

In [ ]:
plot_blood_draw_statistics(df, png="Blood Draw Proportions")

In [ ]:
plot_correlations(df)

In [ ]:
blood_df, clinical_df = split_csv(df)